# Importing Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import gc

# Connection to drive

In [ ]:
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running on Google Colab. ")
except:
    IN_COLAB = False
    print("Not running on Google Colab. ")

In [3]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

In [4]:
if IN_COLAB:
    os.chdir('/content/gdrive/MyDrive/Tesi/dataset')
else:
    os.chdir('./dataset')

# Downloading, Reading and Merging Data

In [5]:
def move_directories(orig_path, dest_path):
    # Check if the destination folder exists, otherwise create it
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    
    # Iter over all files and folders in the source directory
    for item in os.listdir(orig_path):
        orig_item_path = os.path.join(orig_path, item)
        
        # Check if it is a folder
        if os.path.isdir(orig_item_path):
            # Defines the destination path for the folder
            dest_item_path = os.path.join(dest_path, item)
            
            # Move the folder by renaming it
            os.rename(orig_item_path, dest_item_path)
            print(f"Moved directory: {orig_item_path} -> {dest_item_path}")

In [ ]:
import kagglehub

# Check if some subdirectories are not empty
path_to_check = "./LUFlow/"
subdirs = [d for d in os.listdir(path_to_check) if os.path.isdir(os.path.join(path_to_check, d))]
non_empty_subdirs = [d for d in subdirs if os.listdir(os.path.join(path_to_check, d))]

dest_path = './LUFlow'

if non_empty_subdirs:
    print("Non-empty subdirectories:", non_empty_subdirs)
    print("Skip downloading.")
else:
    print("All subdirectories are empty.")
    print("Download dataset.")

    # Download latest version
    path = kagglehub.dataset_download("mryanm/luflow-network-intrusion-detection-data-set")

    print("Path to dataset files:", path)

    move_directories(path, dest_path)

In [ ]:
encoded_dataset = './LUFlow/encoded_dataset.csv'

if os.path.isfile(encoded_dataset):
    print("Encoded dataset is in the directory.")
    ENCODED = True
else:
    print("Encoded dataset is not in the directory.")
    ENCODED = False

In [8]:
if not ENCODED:

    df_list = []
    i = 0

    selected_columns = ['bytes_in', 'bytes_out', 'num_pkts_out', 'num_pkts_in', 'proto', 'time_start', 'label']
    dtype_dict = {'bytes_in': np.int32, 'bytes_out': np.int32, 'num_pkts_out': np.int32, 
                  'num_pkts_in': np.int32, 'proto': np.int32, 'time_start': np.int64, 'label': str}

    for root, dirs, files in os.walk(dest_path):
        for file in files:
            # checks if 'file' does not exist in the directory
            # checks if 'csv' is in the file name
            # checks if a particular string is in the file name
            if not os.path.isfile(file) and 'csv' in file:
                chunk_iter = pd.read_csv(os.path.join(root, file), chunksize=10000, usecols=selected_columns, dtype=dtype_dict)
                for chunk in chunk_iter:
                    df_list.append(chunk)
                
                del chunk_iter
                gc.collect()
                i += 1

    df = pd.concat(df_list, ignore_index=True)

    del df_list
    gc.collect()

In [9]:
# sorted_dataset = './LUFlow/sorted_encoded_dataset.csv'

# if os.path.isfile(sorted_dataset) and ENCODED:
#     print("Sorted dataset is in the directory.")
# elif ENCODED:
#     print("Sorting dataset.")

#     dataset_encoded = pd.read_csv(encoded_dataset)
#     dataset_encoded_sorted = dataset_encoded.sort_values(by=['time_start'])

#     dataset_encoded_sorted.to_csv(sorted_dataset, index=False)

In [10]:
QUART = True

subset = 500

if ENCODED and QUART:
    import pandas as pd

    # Count number of lines in the file
    # with open(sorted_dataset) as f:
    with open(encoded_dataset) as f:
        total_rows = sum(1 for _ in f) - 1  # exclude header

    # Compute the fraction
    n_quarter_rows = total_rows // subset

    # Load only first quarter of the dataset
    # df = pd.read_csv(sorted_dataset, nrows=n_quarter_rows)
    df = pd.read_csv(encoded_dataset, nrows=n_quarter_rows)

# Exploratory Data Analysis

In [11]:
import matplotlib.pyplot as plt

if not ENCODED:
    # Count the occurrences of each label
    label_counts = df['label'].value_counts()

    plt.figure(figsize=(8, 8))
    plt.pie(label_counts, labels=label_counts.index, autopct=lambda p: '{:.0f}\n({:.1f}%)'.format(p * sum(label_counts) / 100, p))

    # Show the plot
    plt.title('Distribution of Labels')
    plt.show()

# Data Preprocessing

In [ ]:
df.dropna(axis=1, inplace = True)

missing_values = df.isnull().sum()
print(missing_values)

In [13]:
if not ENCODED:
    df = df[df['label'].isin(['benign', 'malicious'])].copy()

    df = df.reset_index(drop=True)

In [14]:
from sklearn.preprocessing import LabelEncoder

if not ENCODED:
    label_encoder = LabelEncoder()

    # Fit and transform the label column
    df['label'] = label_encoder.fit_transform(df['label'])

In [15]:
if not ENCODED:
    df.to_csv(encoded_dataset, index=False)

## Construction of Interval Information Granules

### Selecting index using time windows

In [16]:
df.loc[df['time_start'].notna() & (df['time_start'] > 1e12), 'time_start'] //= 1000
df = df.sort_values(by=['time_start'])

In [17]:
perc = 0.8

rows_perc = int(len(df)*perc)

df_train = df.iloc[:rows_perc]

In [18]:
if os.path.exists('./LUFlow/GranulatedData'):
    GRANULATED = True
else:
    GRANULATED = False

In [ ]:
from tqdm import tqdm

# time_slices = [16, 128, 256, 1024, 2048, 3016]
time_slices = [1, 8, 16, 32, 64, 128]

if not GRANULATED:
  matching_indices_slices = {}

  time_start_values = df_train['time_start'].values
  n = time_start_values

  for ts in tqdm(time_slices):
    ts_milli = ts * 1000
    # maximum time limit of the granule
    limit = time_start_values[0] + ts_milli

    # number of groups for time_slice
    num_group = 0
    group_dict = {num_group:[0]}

    for i in range(1, len(n)):
        # if the value of time_start is greater than limit then limit is updated and
        # it is initilized a new group otherwise the index of the row is added to group_dict
        if time_start_values[i] >= limit:
          limit = time_start_values[i] + ts_milli
          num_group += 1

          group_dict[num_group] = [i]
        else:
          group_dict[num_group].append(i)
    
    matching_indices_slices[ts] = group_dict
    del group_dict
    gc.collect()

### Drop Label column

In [ ]:
# Separate features (X) and target variable (y)
X_train = df_train.loc[:, df_train.columns != 'label']
y_train = df_train['label']
y_train = y_train.to_frame()

del df, df_train
gc.collect()

### Creating interval information granules with both approach to compute time spent creating granules


In [ ]:
import math
import time

columns = ['num_frames_clust', 'time_clust', 'num_frames_just', 'time_just']

if not GRANULATED:
  # alpha = 2.0
  # alpha = 1.0
  # alpha = 0.5
  # alpha = 0.1
  # alpha = 0.01
  alpha = 0.01
  l = 100

  a_int = int(alpha)
  
  for time_slice in time_slices:
    
    df = pd.DataFrame(columns=columns)
    df.to_csv(f'../time_comparison_{subset}_{time_slice}.csv', index=False)

    for group in tqdm(matching_indices_slices[time_slice]):

      group_indeces = matching_indices_slices[time_slice][group]

      group_data = X_train.iloc[group_indeces]

      N = len(group_data)

      if N == 1:
        granule_data_y = y_train.loc[group_data.index]

        # Time spent by Justifiable Granularity
        start_just = time.time()
        row = group_data.iloc[0]
        row['label'] = granule_data_y['label'].iloc[0]
        row = row.to_frame().T
        end_just = time.time()

        # Time spent by Clustering
        start_clust = time.time()
        row = group_data.iloc[0]
        row['label'] = granule_data_y['label'].iloc[0]
        end_clust = time.time()

        time_spent_just = end_just - start_just
        time_spent_clust = end_clust - start_clust

        len_just = N
        len_clust = N

        time_row = {'num_frames_clust': len_clust, 'time_clust': time_spent_clust, 'num_frames_just': len_just, 'time_just': time_spent_just}
        time_row = pd.DataFrame([time_row])
        time_row.to_csv(f'../time_comparison_{subset}_{time_slice}.csv', mode='a', header=False, index=False)

        continue
      
      stats = group_data['time_start'].agg(['min', 'max', 'mean'])
      y_min, y_max, m = stats['min'], stats['max'], stats['mean']

      y_max_rel = y_max - y_min

      m_rel = m - y_min

      delta_y = (y_max_rel - m_rel)/l

      V_b_opt, V_a_opt = float('-inf'), float('-inf')
      a_opt, b_opt = None, None

      for h in range(0, l):
        b_rel = m_rel+h*delta_y
        a_rel = m_rel-h*delta_y

        b = y_min + b_rel
        sp_b = math.exp(-alpha*abs(m_rel-b_rel))

        a = y_min + a_rel
        sp_a = math.exp(-alpha*abs(a_rel-m_rel))

        cov_b = ((group_data['time_start'] > m) & (group_data['time_start'] <= b)).sum() / N
        cov_a = ((group_data['time_start'] > a) & (group_data['time_start'] <= m)).sum() / N

        V_b = cov_b*sp_b
        V_a = cov_a*sp_a

        if V_a > V_a_opt:
          a_opt, V_a_opt = a, V_a

        if V_b > V_b_opt:
          b_opt, V_b_opt = b, V_b

      condition = (group_data['time_start'] > a_opt) & (group_data['time_start'] <= b_opt)
      filtered_granule = group_data[condition]

      if filtered_granule.empty:

        granule_data_y = y_train.loc[group_data.index]['label']

        # DA DEFINIRE COME TRATTARE IN QUESTO CASO JUSTIFIABLE GRANULARITY
        # Time spent by Justifiable Granularity
        start_just = time.time()
        row = group_data.iloc[0]
        row['label'] = granule_data_y.iloc[0]
        # row['label'] = granule_data_y.iat[0]
        end_just = time.time()
        
        # Time spent by Clustering
        # granule_data_y = y_train.loc[group_indeces]
        start_clust = time.time()
        row = group_data[['bytes_in', 'bytes_out', 'num_pkts_out', 'num_pkts_in', 'proto']].sum()
        row['time_start'] = group_data['time_start'].iloc[0]
        row['label'] = granule_data_y.mode().iloc[0]
        end_clust = time.time()

        # Compute the time spent for each method and save to csv
        time_spent_just = end_just - start_just
        time_spent_clust = end_clust - start_clust

        len_just = 1
        len_clust = N

        time_row = {'num_frames_clust': len_clust, 'time_clust': time_spent_clust, 'num_frames_just': len_just, 'time_just': time_spent_just}
        time_row = pd.DataFrame([time_row])
        time_row.to_csv(f'../time_comparison_{subset}_{time_slice}.csv', mode='a', header=False, index=False)

        continue  

      selected_rows = y_train.loc[filtered_granule.index]

      # Time spent by Justifiable Granularity
      start_just = time.time()
      # if len(filtered_granule) == 1:
      #   row = filtered_granule.iloc[0]
      # else:
      row = filtered_granule[['bytes_in', 'bytes_out', 'num_pkts_out', 'num_pkts_in', 'proto']].sum()      
      row['time_start'] = filtered_granule['time_start'].iloc[0]
      row['label'] = selected_rows.mode().iloc[0]
      end_just = time.time()

      # Time spent by Clustering
      granule_data_y = y_train.loc[group_data.index]['label']
      start_clust = time.time()
      row = group_data[['bytes_in', 'bytes_out', 'num_pkts_out', 'num_pkts_in', 'proto']].sum()
      row['time_start'] = group_data['time_start'].iloc[0]
      row['label'] = granule_data_y.mode().iloc[0]
      end_clust = time.time()

      # Compute the time spent for each method and save to csv
      time_spent_just = end_just - start_just
      time_spent_clust = end_clust - start_clust

      len_just = len(filtered_granule)
      len_clust = N

      time_row = {'num_frames_clust': len_clust, 'time_clust': time_spent_clust, 'num_frames_just': len_just, 'time_just': time_spent_just}
      time_row = pd.DataFrame([time_row])
      time_row.to_csv(f'../time_comparison_{subset}_{time_slice}.csv', mode='a', header=False, index=False)
      # print(time_row)

      # del group_data, filtered_granule, selected_rows

In [22]:
# a_int = int(alpha)
mean_file = f'../mean_{subset}.csv'
std_file = f'../std_{subset}.csv'

columns = ['num_frames_clust', 'time_clust', 'num_frames_just', 'time_just', 'time_slice', 'perc_red']

df_col = pd.DataFrame(columns=columns)
df_col.to_csv(mean_file, index=False)
df_col.to_csv(std_file, index=False)


for time_slice in time_slices:
    df = pd.read_csv(f'../time_comparison_{subset}_{time_slice}.csv')

    mean = df.mean()
    std = df.std()

    mean = mean.to_frame().T
    std = std.to_frame().T

    mean['time_slice'] = time_slice
    std['time_slice'] = time_slice

    perc_red = ((df['num_frames_clust'] - df['num_frames_just']) / df['num_frames_clust']) * 100
    perc_red_mean = perc_red.mean()
    perc_red_std = perc_red.std()

    mean['perc_red'] = perc_red_mean
    std['perc_red'] = perc_red_std

    mean.to_csv(mean_file, mode='a', header=False, index=False)
    std.to_csv(std_file, mode='a', header=False, index=False)

In [ ]:
x_axis = np.arange(len(time_slices))
figure, axis = plt.subplots(2, 2, figsize=(12, 8))

mean_df = pd.read_csv(mean_file)
std_df = pd.read_csv(std_file)

axis[0, 0].plot(x_axis, mean_df['time_clust'], label='Clustering')
axis[0, 0].plot(x_axis, mean_df['time_just'], label='Justifiable Granularity')
axis[0, 0].set_title('Mean Time Comparison')
axis[0, 0].set_xticks(x_axis)  
axis[0, 0].set_xticklabels(time_slices) 
axis[0, 0].set_xlabel('Time Slice')

axis[0, 1].plot(x_axis, std_df['time_clust'], label='Clustering')
axis[0, 1].plot(x_axis, std_df['time_just'], label='Justifiable Granularity')
axis[0, 1].set_title('Standard Deviation Time Comparison')
axis[0, 1].set_xticks(x_axis)  
axis[0, 1].set_xticklabels(time_slices) 
axis[0, 1].set_xlabel('Time Slice')

axis[1, 0].plot(x_axis, mean_df['num_frames_clust'], label='Clustering')
axis[1, 0].plot(x_axis, mean_df['num_frames_just'], label='Justifiable Granularity')
axis[1, 0].set_title('Mean of frames number')
axis[1, 0].set_xticks(x_axis)  
axis[1, 0].set_xticklabels(time_slices) 
axis[1, 0].set_xlabel('Time Slice')

axis[1, 1].plot(x_axis, std_df['num_frames_clust'], label='Clustering')
axis[1, 1].plot(x_axis, std_df['num_frames_just'], label='Justifiable Granularity')
axis[1, 1].set_title('Standard Deviation of frames number')
axis[1, 1].set_xticks(x_axis)  
axis[1, 1].set_xticklabels(time_slices) 
axis[1, 1].set_xlabel('Time Slice')

plt.tight_layout() # rect=[0, 0.1, 1, 1]
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

lines, labels = [], []
for ax in axis.flat:
    for line, label in zip(*ax.get_legend_handles_labels()):
        if label not in labels:  # Avoid duplicate labels
            lines.append(line)
            labels.append(label)

figure.legend(lines, labels, loc='lower center', ncol=2, fontsize='medium', bbox_to_anchor=(0.5, -0.05))


plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for time_slice in time_slices:
    df = pd.read_csv(f'../time_comparison_{subset}_{time_slice}.csv')
    X_axis = np.arange(len(df))
    plt.plot(X_axis, df['time_clust'], label='Clustering')
    plt.plot(X_axis, df['time_just'], label='Justifiable Granularity')

    plt.xlabel('Granule')
    plt.ylabel('Time (s)')
    plt.title(f'Time comparison for time slice {time_slice}')
    plt.legend()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for time_slice in time_slices:
    df = pd.read_csv(f'../time_comparison_{subset}_{time_slice}.csv')
    X_axis = np.arange(len(df))
    plt.plot(X_axis, df['num_frames_clust'], label='Clustering')
    plt.plot(X_axis, df['num_frames_just'], label='Justifiable Granularity')

    plt.xlabel('Granule')
    plt.ylabel('Number of frames')
    plt.title(f'Number of frames comparison for time slice {time_slice}')
    plt.legend()
    plt.show()